In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
fake = pd.read_csv('D:\\6th Sem\\Project- Fake News Detection\\Python Model FN\\Fake.csv',encoding='latin1')
real = pd.read_csv('D:\\6th Sem\\Project- Fake News Detection\\Python Model FN\\True.csv',encoding='latin1')

In [3]:
fake.head(5)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Yearâ...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obamaâs Na...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
real.head(5)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,31-Dec-17
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,29-Dec-17
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,31-Dec-17
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,30-Dec-17
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,29-Dec-17


In [5]:
fake['target'] = 0
real['target'] = 1

In [6]:
#Combining Title and Text
real["text"] = real["title"] + " " + real["text"]
fake["text"] = fake["title"] + " " + fake["text"]

In [7]:
real["text"]

0        As U.S. budget fight looms, Republicans flip t...
1        U.S. military to accept transgender recruits o...
2        Senior U.S. Republican senator: 'Let Mr. Muell...
3        FBI Russia probe helped by Australian diplomat...
4        Trump wants Postal Service to charge 'much mor...
                               ...                        
21412    'Fully committed' NATO backs new U.S. approach...
21413    LexisNexis withdrew two products from Chinese ...
21414    Minsk cultural hub becomes haven from authorit...
21415    Vatican upbeat on possibility of Pope Francis ...
21416    Indonesia to buy $1.14 billion worth of Russia...
Name: text, Length: 21417, dtype: object

In [8]:
real = real.drop(["subject", "date"], axis=1)
fake = fake.drop(["subject", "date"], axis=1)

In [9]:
real

,title,text,target
0,"As U.S. budget fight looms, Republicans flip t...","As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,Trump wants Postal Service to charge 'much mor...,1
...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,'Fully committed' NATO backs new U.S. approach...,1
21413,LexisNexis withdrew two products from Chinese ...,LexisNexis withdrew two products from Chinese ...,1
21414,Minsk cultural hub becomes haven from authorities,Minsk cultural hub becomes haven from authorit...,1
21415,Vatican upbeat on possibility of Pope Francis ...,Vatican upbeat on possibility of Pope Francis ...,1


In [10]:
data = real.append(fake, ignore_index=True)
del real, fake

In [11]:
data

,title,text,target
0,"As U.S. budget fight looms, Republicans flip t...","As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,Trump wants Postal Service to charge 'much mor...,1
...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,McPain: John McCain Furious That Iran Treated ...,0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,0
44895,Sunnistan: US and Allied âSafe Zoneâ Plan ...,Sunnistan: US and Allied âSafe Zoneâ Plan ...,0
44896,How to Blow $700 Million: Al Jazeera America F...,How to Blow $700 Million: Al Jazeera America F...,0


In [12]:
from sklearn.utils import shuffle
data = shuffle(data)
data = data.reset_index(drop=True)
data.head()

,title,text,target
0,Jill Stein Forced To Drop Pennsylvania Recoun...,Jill Stein Forced To Drop Pennsylvania Recoun...,0
1,WOW! FBI SUED Over Andrew Breitbart Records Re...,WOW! FBI SUED Over Andrew Breitbart Records Re...,0
2,Suing governments over terror no sure thing de...,Suing governments over terror no sure thing de...,1
3,TEXAS CIVIL RIGHTS LAWYER and Dartmouth Grad S...,TEXAS CIVIL RIGHTS LAWYER and Dartmouth Grad S...,0
4,Trump Vineyard Asks Labor Department For More...,Trump Vineyard Asks Labor Department For More...,0


In [13]:
data.isnull().any()

title     False
text      False
target    False
dtype: bool

In [14]:
data['text'] = data['text'].apply(lambda x: x.lower())
data

,title,text,target
0,Jill Stein Forced To Drop Pennsylvania Recoun...,jill stein forced to drop pennsylvania recoun...,0
1,WOW! FBI SUED Over Andrew Breitbart Records Re...,wow! fbi sued over andrew breitbart records re...,0
2,Suing governments over terror no sure thing de...,suing governments over terror no sure thing de...,1
3,TEXAS CIVIL RIGHTS LAWYER and Dartmouth Grad S...,texas civil rights lawyer and dartmouth grad s...,0
4,Trump Vineyard Asks Labor Department For More...,trump vineyard asks labor department for more...,0
...,...,...,...
44893,"Obama congratulates Clinton, to meet with Sand...","obama congratulates clinton, to meet with sand...",1
44894,"âSelf-Fundingâ Trump Gets Desperate, Sham...","âself-fundingâ trump gets desperate, sham...",0
44895,Trump budget cuts may stir backlash in rural A...,trump budget cuts may stir backlash in rural a...,1
44896,Illinois governor backs 'Democratic' state pen...,illinois governor backs 'democratic' state pen...,1


In [15]:
import string
def punctuation_removal(text):
    all_list = [char for char in text if char not in string.punctuation]
    clean_str = ''.join(all_list)
    return clean_str
data['text'] = data['text'].apply(punctuation_removal)
data

,title,text,target
0,Jill Stein Forced To Drop Pennsylvania Recoun...,jill stein forced to drop pennsylvania recoun...,0
1,WOW! FBI SUED Over Andrew Breitbart Records Re...,wow fbi sued over andrew breitbart records req...,0
2,Suing governments over terror no sure thing de...,suing governments over terror no sure thing de...,1
3,TEXAS CIVIL RIGHTS LAWYER and Dartmouth Grad S...,texas civil rights lawyer and dartmouth grad s...,0
4,Trump Vineyard Asks Labor Department For More...,trump vineyard asks labor department for more...,0
...,...,...,...
44893,"Obama congratulates Clinton, to meet with Sand...",obama congratulates clinton to meet with sande...,1
44894,"âSelf-Fundingâ Trump Gets Desperate, Sham...",âselffundingâ trump gets desperate shamel...,0
44895,Trump budget cuts may stir backlash in rural A...,trump budget cuts may stir backlash in rural a...,1
44896,Illinois governor backs 'Democratic' state pen...,illinois governor backs democratic state pensi...,1


In [16]:
from collections import Counter
all_text = ' '.join(data.title.values)
words = all_text.split()
u_words = Counter(words).most_common()
u_words_counter = u_words
u_words_frequent = [word[0] for word in u_words if word[1]>5]

In [17]:
import matplotlib.pyplot as plt

import re

from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, LSTM, Embedding, Reshape
from keras.models import load_model, model_from_json

from sklearn.model_selection import train_test_split

import os
import urllib

from urllib.request import urlretrieve

from os import mkdir, makedirs, remove, listdir

from collections import Counter


with open('./dataglove/glove.6B.50d.txt','rb') as f:
    lines = f.readlines()
    
glove_weights = np.zeros((len(lines), 50))
words = []
for i, line in enumerate(lines):
    word_weights = line.split()
    words.append(word_weights[0])
    weight = word_weights[1:]
    glove_weights[i] = np.array([float(w) for w in weight])
word_vocab = [w.decode("utf-8") for w in words]

word2glove = dict(zip(word_vocab, glove_weights))

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\Users\HZT\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    fp, pathname, description = imp.find_module('_pywrap_tensorflow_internal', [dirname(__file__)])
  File "C:\Users\HZT\anaconda3\lib\imp.py", line 296, in find_module
    raise ImportError(_ERR_MSG.format(name), name=name)
ImportError: No module named '_pywrap_tensorflow_internal'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\HZT\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\HZT\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\HZT\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow